In [2]:
import pandas as pd 
import matplotlib as plt
import re

In [3]:
laptops_data = pd.read_csv("laptops.csv",encoding='latin-1')
laptops_data = laptops_data.drop(index=1191)
laptops_data.to_csv('laptops_samsung.csv')


#Ram Cleaning
ram = laptops_data['RAM']
ram = ram.str.replace(r'(GB)', '')
ram.columns = ['RAM']
ram
print(laptops_data.iloc[[1191]])

     Manufacturer Model Name  Category Screen Size    Screen  \
1192           HP     250 G6  Notebook       15.6"  1366x768   

                             CPU  RAM    Storage                    GPU  \
1192  Intel Core i5 7200U 2.5GHz  4GB  500GB HDD  Intel HD Graphics 620   

     Operating System Operating System Version  Weight Price (Euros)  
1192          Windows                       10  1.86kg        485,00  


In [4]:

#CPU CLEANING
intel = laptops_data[laptops_data['CPU'].str.match('Intel')]['CPU']
amd= laptops_data[laptops_data['CPU'].str.match('AMD')]['CPU'].str.split(' ', expand= True)
amd.columns = ['Brand', 'Series','Model','Clockspeed']
intelnew = pd.DataFrame(columns=['Brand','Series','Clockspeed'])


data = []

# print(list(intelnew.columns.values))
# print(list(intel))


def intel_brand_cleaner(x):
    for i in x:
        brand = i.split()[0]
        series = re.findall(r'(?<=Intel )\w+ *[a-zA-Z0-9_.-]* *[a-zA-Z0-9_.-]* *[a-zA-Z0-9_.-]* *[a-zA-Z0-9_.-]*(?= \d.*\d*GHz)', i)
        clockspeed = re.findall(r'(?<= )\d{1}.?\d*(?=GHz)', i)
        zipped = zip(list(intelnew.columns.values),[brand,series[0],clockspeed[0]])
        data.append(dict(zipped))
   
        
intel_brand_cleaner(list(intel))
intelnew = intelnew.append(data)
intelnew.index = intel.index
intelnew
cpu = pd.concat([intelnew, amd],axis=1,sort=True)

print(cpu.iloc[1190])



Brand                  Intel
Series        Core i5 7300HQ
Clockspeed               2.5
Brand                    NaN
Series                   NaN
Model                    NaN
Clockspeed               NaN
Name: 1190, dtype: object


In [5]:
#Ram Cleaning
data = []
screen = pd.DataFrame(columns=['Resolution','Screen_Type'])
def clean_screen(x):
    for i in x:
        resolution = re.findall(r'\d{4}x\d+', i)
        screenType = re.findall(r'(Touchscreen)', i)
        if screenType == []:
            zipped = zip(list(screen.columns.values),[resolution[0],'No Touchscreen'])
        else:
            zipped = zip(list(screen.columns.values),[resolution[0],'Touchscreen'])
        data.append(dict(zipped))
clean_screen(list(laptops_data['Screen']))
screen = screen.append(data)



In [6]:
#Storage Cleaner
data = []
storage = pd.DataFrame(columns=['SSD','HDD','HYBRID','SSD_CAPACITY','HDD_CAPACITY','HYBRID_CAPACITY','TOTAL_STORAGE'])
ab = 0
c = 1 if ab == 0 else 2

def clean_storage(x):
    for i in x:
        
        ssd_capacity = re.findall(r'\d+GB(?= SSD| Flash Storage)|\d+TB(?= SSD| Flash Storage)', i)[0] if re.findall(r'\d+GB(?= SSD| Flash Storage)|\d+TB(?= SSD| Flash Storage)', i)!= [] else '0'
        ssd_capacity = (1000 * int(re.findall(r'\d+',ssd_capacity)[0])) if re.match('\d+TB',ssd_capacity)!= None else int(re.findall(r'\d+',ssd_capacity)[0])
        ssd = "True" if ssd_capacity != 0 else "False"
        
        hdd_capacity = re.findall(r'\d+GB(?= HDD)|\d+TB(?= HDD)', i)[0] if re.findall(r'\d+GB(?= HDD)|\d+TB(?= HDD)', i)!= [] else '0'
        hdd_capacity = (1000 * int(re.findall(r'\d+',hdd_capacity)[0])) if re.match('\d+TB',hdd_capacity)!= None else int(re.findall(r'\d+',hdd_capacity)[0])
        hdd = "True" if hdd_capacity != 0 else "False"
       
        hybrid_capacity = re.findall(r'\d+GB(?= Hybrid)|\d+TB(?= Hybrid)', i)[0] if re.findall(r'\d+GB(?= Hybrid)|\d+TB(?= Hybrid)', i)!= [] else '0'
        hybrid_capacity = (1000 * int(re.findall(r'\d+',hybrid_capacity)[0])) if re.match('\d+TB',hybrid_capacity)!= None else int(re.findall(r'\d+',hybrid_capacity)[0])
        hybrid = "True" if hybrid_capacity != 0 else "False"
        total_storage = sum([ssd_capacity,hdd_capacity,hybrid_capacity])
        zipped = zip(list(storage.columns.values),[ssd,hdd,hybrid,ssd_capacity,hdd_capacity,hybrid_capacity,total_storage])
        data.append(dict(zipped))
clean_storage(list(laptops_data[' Storage']))
storage = storage.append(data)
storage
storage

,SSD,HDD,HYBRID,SSD_CAPACITY,HDD_CAPACITY,HYBRID_CAPACITY,TOTAL_STORAGE
0,True,False,False,128,0,0,128
1,True,False,False,128,0,0,128
2,True,False,False,256,0,0,256
3,True,False,False,512,0,0,512
4,True,False,False,256,0,0,256
...,...,...,...,...,...,...,...
1297,True,False,False,128,0,0,128
1298,True,False,False,512,0,0,512
1299,True,False,False,64,0,0,64
1300,False,True,False,0,1000,0,1000


In [7]:
#Combining Everrything
dataframelist = [laptops_data[['Manufacturer','Model Name','Category','Screen Size','GPU','Operating System','Operating System Version','Weight'
 ,'Price (Euros)']],screen,ram,storage]
result = pd.concat(dataframelist,axis=1)
result.to_csv('laptops_cleaned.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1303 entries, 0 to 1302
Data columns (total 19 columns):
Manufacturer                1302 non-null object
Model Name                  1302 non-null object
Category                    1302 non-null object
Screen Size                 1302 non-null object
GPU                         1302 non-null object
Operating System            1302 non-null object
Operating System Version    1133 non-null object
Weight                      1302 non-null object
Price (Euros)               1302 non-null object
Resolution                  1302 non-null object
Screen_Type                 1302 non-null object
RAM                         1302 non-null object
SSD                         1302 non-null object
HDD                         1302 non-null object
HYBRID                      1302 non-null object
SSD_CAPACITY                1302 non-null object
HDD_CAPACITY                1302 non-null object
HYBRID_CAPACITY             1302 non-null object
TOTAL_STORA

In [7]:
laptops_data[['Manufacturer','Model Name','Category','Screen Size','GPU','Operating System','Operating System Version','Weight'
 ,'Price (Euros)']]


,Manufacturer,Model Name,Category,Screen Size,GPU,Operating System,Operating System Version,Weight,Price (Euros)
0,Apple,MacBook Pro,Ultrabook,"13.3""",Intel Iris Plus Graphics 640,macOS,NaN,1.37kg,"1339,69"
1,Apple,Macbook Air,Ultrabook,"13.3""",Intel HD Graphics 6000,macOS,NaN,1.34kg,"898,94"
2,HP,250 G6,Notebook,"15.6""",Intel HD Graphics 620,No OS,NaN,1.86kg,"575,00"
3,Apple,MacBook Pro,Ultrabook,"15.4""",AMD Radeon Pro 455,macOS,NaN,1.83kg,"2537,45"
4,Apple,MacBook Pro,Ultrabook,"13.3""",Intel Iris Plus Graphics 650,macOS,NaN,1.37kg,"1803,60"
...,...,...,...,...,...,...,...,...,...
1298,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,"14.0""",Intel HD Graphics 520,Windows,10,1.8kg,"638,00"
1299,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,"13.3""",Intel HD Graphics 520,Windows,10,1.3kg,"1499,00"
1300,Lenovo,IdeaPad 100S-14IBR,Notebook,"14.0""",Intel HD Graphics,Windows,10,1.5kg,"229,00"
1301,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,"15.6""",AMD Radeon R5 M330,Windows,10,2.19kg,"764,00"


In [14]:
laptops_cleaned = pd.read_csv("supa_clean_data.csv")
laptops_cleaned.loc[laptops_cleaned['Category'] == "Ultrabook"]


,Manufacturer,Model Name,Category,Screen Size,GPU,Price (Euros),Screen_Type,RAM,SSD,HDD,...,HDD_CAPACITY,HYBRID_CAPACITY,TOTAL_STORAGE,Brand,Clockspeed,Model,Series,Width,Height,fullRes
0,Apple,MacBook Pro,Ultrabook,13.3,Intel Iris Plus Graphics 640,1339,No Touchscreen,8,True,False,...,0,0,128,Intel,2.3,NaN,Core i5,2560,1600,4096000
1,Apple,Macbook Air,Ultrabook,13.3,Intel HD Graphics 6000,898,No Touchscreen,8,True,False,...,0,0,128,Intel,1.8,NaN,Core i5,1440,900,1296000
3,Apple,MacBook Pro,Ultrabook,15.4,AMD Radeon Pro 455,2537,No Touchscreen,16,True,False,...,0,0,512,Intel,2.7,NaN,Core i7,2880,1800,5184000
4,Apple,MacBook Pro,Ultrabook,13.3,Intel Iris Plus Graphics 650,1803,No Touchscreen,8,True,False,...,0,0,256,Intel,3.1,NaN,Core i5,2560,1600,4096000
6,Apple,MacBook Pro,Ultrabook,15.4,Intel Iris Pro Graphics,2139,No Touchscreen,16,True,False,...,0,0,256,Intel,2.2,NaN,Core i7,2880,1800,5184000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,HP,EliteBook 840,Ultrabook,14.0,Intel HD Graphics 520,2198,No Touchscreen,8,True,False,...,0,0,256,Intel,2.5,NaN,Core i7 6500U,2560,1440,3686400
1240,Asus,ZenBook UX310UA-WB71,Ultrabook,13.3,Intel HD Graphics 520,1280,No Touchscreen,8,True,False,...,0,0,256,Intel,2.5,NaN,Core i7 6500U,1920,1080,2073600
1260,Asus,ZenBook UX305CA-UBM1,Ultrabook,13.3,Intel HD Graphics 515,729,No Touchscreen,8,True,False,...,0,0,512,Intel,0.9,NaN,Core M 6Y30,1920,1080,2073600
1274,Asus,ZenBook UX305CA-UBM1,Ultrabook,13.3,Intel HD Graphics 515,729,No Touchscreen,8,True,False,...,0,0,512,Intel,0.9,NaN,Core M 6Y30,1920,1080,2073600
